In [21]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sc
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

def dataset(dataFile, labelFile):
    dataFile = "./communication/" + dataFile
    labelFile = "./communication/" + labelFile
    my_data = sc.loadmat(dataFile)
    my_labels = sc.loadmat(labelFile)
    my_data = my_data['Y']
    X = my_labels['L_S_x']
    myOrig = table_data(my_data, my_labels['L_Constellations'][0], X)
    myTable = assign_labels(myOrig)
    return myTable

def table_data(my_data, cons, label):
    block = my_data.shape[1]
    my_data_size = my_data.shape[0] * block
    my_data_div = my_data.T.reshape(my_data_size,)
    cons_array = np.array([[cons[i]]*my_data.shape[0] for i in range(0,block)]).reshape(my_data_size,)
    block_array = np.array([([i+1]*my_data.shape[0])for i in range(0, block)]).reshape(my_data_size,)
    label_array = label.T.reshape(my_data_size,)
    test_pd = pd.DataFrame({'real':my_data_div.real,'imag':my_data_div.imag,
            'cons':cons_array, 'block':block_array,
            'label':label_array})
    return test_pd

def assign_labels(myTable):
    myTest = myTable.copy()
    myTest.loc[myTest.cons==2, 'label'] = myTest.loc[myTest.cons==2, 'label']+4
    myTest.label = myTest.label-1
    return myTest


from numpy.random import default_rng
def training_set(myTable):
    block = myTable.shape[0]
    rng = default_rng()
    sample_size = int(0.8 * block)
    numbers = rng.choice(range(1, block + 1), size=sample_size, replace=False)
    training_dataset = myTable[myTable.block.isin(numbers)]
    return training_dataset

def test_set(myTable, training_dataset):
    remaining = myTable.drop(training_dataset.index)
    return remaining



def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(50, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(20)
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(0.01),
                 metrics=['accuracy'])
    return model

def get_total_loss(predict_label, true_label):
    i = 0
    for j in range(len(predict_label)):
        prediction = np.argmax(predict_label[j])
        if prediction != true_label[j]:
            i = i + 1
    return i/len(predict_label)

datas = "INR30data"
labels = "INR30labels"


def get_training(myTable, epochs, files):
    train_dataset = training_set(myTable)
    test_dataset = test_set(myTable, train_dataset)
    train_features = train_dataset.copy()
    test_features = test_dataset.copy()
    train_labels = pd.DataFrame([train_features.pop('label')]).T
    test_labels = pd.DataFrame([test_features.pop('label')]).T
    normalizer = preprocessing.Normalization()
    normalizer.adapt(np.array(train_features))
    dnn_real_model = build_and_compile_model(normalizer)
    history = dnn_real_model.fit(
        train_features, train_labels,
        validation_split=0.2,
        epochs=epochs)
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    test_results = {}
    test_results['signal'] = dnn_real_model.evaluate(
    test_features,
    test_labels, verbose=0)
    probability_model = tf.keras.Sequential([dnn_real_model, 
                                             tf.keras.layers.Softmax()])
    predictions = probability_model.predict(test_features)
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist.tail()
    loss = get_total_loss(predictions, test_labels.to_numpy())
    hist = hist.append(test_results, ignore_index=True)
    hist.to_csv('./results/'+ files + '.csv' , index=False)
    return dnn_real_model


def prediction(model, predictionDataSet, predictionLabels):
    test_dataset = dataset(predictionDataSet, predictionLabels)
    test_features = test_dataset.copy()
    test_labels = pd.DataFrame([test_features.pop('label')]).T
    predictions = model.predict(test_features)
    test_results = {}
    test_results['signal'] = model.evaluate(
    test_features,
    test_labels, verbose=0)
    return test_results
    

In [152]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [Ireal]')
    plt.legend()
    plt.grid(True)

In [206]:
results = dnn_real_model.evaluate(test_features, test_labels)

302/302 [==============================] - 0s 457us/step - loss: 0.3684 - accuracy: 0.7798


In [12]:
myTable = dataset("INR30data", "INR30labels")

In [13]:
datas = "INR30data"
labels = "INR30labels"

In [71]:
model = get_training(new_test, 10, "new_test")

Epoch 1/10
2008/2008 [==============================] - 1s 736us/step - loss: 0.6240 - accuracy: 0.7814 - val_loss: 0.1111 - val_accuracy: 0.9873
Epoch 2/10
2008/2008 [==============================] - 1s 689us/step - loss: 0.4083 - accuracy: 0.8208 - val_loss: 0.0967 - val_accuracy: 0.9681
Epoch 3/10
2008/2008 [==============================] - 1s 658us/step - loss: 0.2922 - accuracy: 0.8633 - val_loss: 0.0228 - val_accuracy: 0.9937
Epoch 4/10
2008/2008 [==============================] - 1s 660us/step - loss: 0.2413 - accuracy: 0.8838 - val_loss: 0.0248 - val_accuracy: 0.9905
Epoch 5/10
2008/2008 [==============================] - 1s 654us/step - loss: 0.2184 - accuracy: 0.8918 - val_loss: 0.0670 - val_accuracy: 0.9801
Epoch 6/10
2008/2008 [==============================] - 1s 663us/step - loss: 0.2127 - accuracy: 0.8958 - val_loss: 0.0065 - val_accuracy: 0.9983
Epoch 7/10
2008/2008 [==============================] - 1s 691us/step - loss: 0.2051 - accuracy: 0.8988 - val_loss: 0.0294 -

In [25]:
anotherTable = dataset("my_data", "my_labels")

In [73]:
prediction(model, "INR30data", "INR30labels")

{'signal': [0.2934909462928772, 0.8456000089645386]}

In [70]:
new_test = pd.concat([myTable,anotherTable], ignore_index=True)

In [75]:
(a+"b") = 1

SyntaxError: can't assign to operator (<ipython-input-75-325436e253a2>, line 1)

In [82]:
   print("I'm Bob.\nWhat's your name?") a

I'm Bob.
What's your name?
